# Pipe 3

This notebook contains scripts for calculating kinematic data.

Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.
Its headers are defined in the code below.

Outputs another `.csv` file _with the **different** headers_.
Those new headers are also defined in the code below.

In [75]:
import csv
import datetime as dt
import math
from geopy.distance import geodesic
from constants import *

In [76]:
INPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/position-csv/"
INPUT_FILENAMES = ["output1.csv"] #, "output2.csv", "output3.csv"]
OUTPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/calc-csv/"

In [77]:
MIN_SATELLITES = 2

In [78]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"

# TODO make this compatible with several levels of precesion
def parseTimestamp(ts: str):
    return dt.datetime.strptime(ts, TIMESTAMP_FORMAT)

The math done in the `calc` method below was originally taken from the `parsefile` method in `rtklib-nav-to-csv.ipynb`.
So far, no adaptions have been made.

In [79]:
def calc(inputPath, outputPath):
    # Create headeres in output path path
    with open(outputPath, 'w') as file:
        writer = csv.DictWriter(file, fieldnames=CALC_CSV_HEADER)
        writer.writeheader()
        file.close()

    lastLat = None
    lastLon = None
    lastVel = None
    lastTs = None

    with open(inputPath, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if int(row["satellites"]) < MIN_SATELLITES: continue

            timestamp = parseTimestamp(row["timestamp"]) 
            lat = float(row["latitude"])
            lon = float(row["longitude"])
            alt = float(row["altitude"])

            sdn = float(row["sdn"])
            sde = float(row["sde"])
            sdu = float(row["sdu"])

            horizAcc = round(math.sqrt(sdn**2 + sde**2), 4) # TODO this is not statistically accurate
            vertAcc = round(sdu, 4) # TODO this may or may not need some math done on it

            # TODO fit some kind of curve to compute a smooth measurement of speed
            if lastLat and lastLon:
                displacement = geodesic((lastLat, lastLon), (lat, lon)).meters # distance in meters
                timediff = (timestamp - lastTs).microseconds / 1000000 # seconds
                groundSpeedRaw = round((displacement/timediff), 4) # m/s

                if lastVel:
                    acceleration = (groundSpeedRaw - lastVel) / (timediff) # m/s^2
                else:
                    acceleration = 0

                lastVel = groundSpeedRaw

                # if acceleration > 100 or groundSpeedRaw > 45:
                #     continue

                speedAcc = 0 # TODO

                with open(outputPath, 'a') as file:
                    csv.DictWriter(file, fieldnames=CALC_CSV_HEADER).writerow({
                        "timestamp": timestamp.strftime(TIMESTAMP_FORMAT),
                        "timestamp_str": timestamp.strftime(TIMESTAMP_FORMAT),
                        "fix_type": row["fix-type"],
                        "latitude": lat,
                        "longitude": lon,
                        "ground_speed_raw": groundSpeedRaw, # m/s
                        "ground_speed_processed": groundSpeedRaw, # m/s, TODO: should be calculated
                        "acceleration": acceleration,
                        "altitude": alt, # meters
                        "horizontal_accuracy": horizAcc,
                        "vertical_accuracy": vertAcc,
                        "speed_accuracy": speedAcc
                    })

            lastLat = lat
            lastLon = lon
            lastTs = timestamp



In [81]:
for name in INPUT_FILENAMES:
    calc(INPUT_FILEPATH+name, OUTPUT_FILEPATH+name)